Notebook to deploy SEA LLM 7B

In [2]:
!pip install -U sagemaker transformers --quiet


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [3]:
import json
import sagemaker
import boto3
from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [4]:
try:
	role = sagemaker.get_execution_role()
except ValueError:
	iam = boto3.client('iam')
	role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

In [5]:
# Hub Model configuration. https://huggingface.co/models
trust_remote_code=True
hub = {
	'HF_MODEL_ID':'SeaLLMs/SeaLLM-7B-v2',
#    'HF_MODEL_TRUST_REMOTE_CODE':json.dumps(trust_remote_code),
	'SM_NUM_GPUS': json.dumps(4)
}

In [6]:
# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
	image_uri=get_huggingface_llm_image_uri("huggingface",version="1.4.0"),
	env=hub,
	role=role,
    transformers_version="4.34.1",
)

In [7]:
# deploy model to SageMaker Inference
predictor = huggingface_model.deploy(
	initial_instance_count=1,
	instance_type="ml.g5.12xlarge",
	container_startup_health_check_timeout=300,
  )

---------!

In [8]:
from transformers import AutoTokenizer

tok = AutoTokenizer.from_pretrained("SeaLLMs/SeaLLM-7B-v2")

# Prompt to generate
messages = [
    { "role": "user", "content": "Pikirkan tahap demi tahap dan jawab dengan bahasa Indonesia. Bila umur Budi ddua kali umur Wati dan jumlah umur mereka adalah dua belas tahun, berapakah umur Budi ?" },
]

# Generation arguments
payload = {
    "do_sample": True,
    "top_p": 0.6,
    "temperature": 0.1,
    "top_k": 50,
    "max_new_tokens": 1024,
    "repetition_penalty": 1.03,
    "return_full_text": False,
    "stop": ["</s>"]
}

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [9]:
# send request
chat = predictor.predict({
  "inputs":tok.apply_chat_template(messages,tokenize=False,add_generation_prompt=True), # convert messages to model input
  "parameters":payload
})

print(chat[0]["generated_text"])

Untuk menyelesaikan masalah ini, kita dapat menggunakan aljabar. Mari kita sebut x sebagai umur Wati dan 2x sebagai umur Budi. Menurut informasi yang diberikan, jumlah umur mereka adalah dua belas tahun, jadi kita dapat menulis persamaan:

x + 2x = 12

Menggabungkan kedua sisi persamaan, kita dapatkan:

3x = 12

Untuk mencari nilai x, kita bagi kedua sisi persamaan dengan 3:

x = 4

Jadi, umur Wati adalah 4 tahun. Karena umur Budi adalah dua kali umur Wati, maka umur Budi adalah:

2 * 4 = 8

Jadi, umur Budi adalah 8 tahun.


In [ ]:
# Cleanup
predictor.delete_model()
predictor.delete_endpoint()
